In [1]:
### Environment setup
import sys
sys.path.append('/content')
from src.fluvius import WaterData, WaterStation
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import geopandas as gpd
import fsspec
from pystac_client import Client
import planetary_computer as pc
import os

import matplotlib.pyplot as plt
# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
# pc.settings.set_subscription_key(<YOUR API Key>)
env_vars = !cat /content/.env

for var in env_vars:
    key, value = var.split(' = ')
    os.environ[key] = value

#################  set up ####################

storage_options={'account_name':os.environ['ACCOUNT_NAME'],\
                 'account_key':os.environ['BLOB_KEY']}
fs = fsspec.filesystem('az',\
                       account_name=storage_options['account_name'],\
                       account_key=storage_options['account_key'])   

In [2]:
import datetime
from src.utils import dates_to_julian

def process_df(df, datatype):
    df = df.copy()
    if datatype == 'ana':
        df = df.rename(columns={'Suspended Sediment Concentration (mg/L)':'SSC (mg/L)',\
                           'Discharge': 'Q (m3/s)'})
    elif datatype == 'itv':
        df = df.rename(columns={'SSC (mg/l)':'SSC (mg/L)',\
                           'Q (m³/s)': 'Q (m3/s)'})
    elif datatype == 'usgs':
        df = df.rename(columns={'Computed instantaneous suspended sediment (mg/L)':'SSC (mg/L)',\
                           'Instantaneous computed discharge (cfs)': 'Q (m3/s)'})
        cfs_to_m3s = 0.028316847
        df['Q (m3/s)'] = cfs_to_m3s * df['Q (m3/s)']
    else:
        df = df.rename(columns={'Instantaneous suspended sediment (mg/L)':'SSC (mg/L)',\
                           'Instantaneous computed discharge (cfs)': 'Q (m3/s)'})
        cfs_to_m3s = 0.028316847
        df['Q (m3/s)'] = cfs_to_m3s * df['Q (m3/s)']
        
    df['julian'] = [dates_to_julian(d) for d in df['Date-Time']]
    selection = ['region', 'site_no', 'sample_id','julian',\
                 'Date-Time', 'Q (m3/s)', 'SSC (mg/L)',\
                 'Chip Cloud Pct','sentinel-2-l2a_R','sentinel-2-l2a_G','sentinel-2-l2a_B']
    return df[selection]
    
container_list = ['ana', 'itv', 'usgsi', 'usgs']
#we are looking for all the individual processed csv
#adding a column for the data type
db = []
for con in container_list:
    f = fs.walk(f'{con}-data/stations')
    processed_list = []
    for a in f:
        for b in a:
            for c in b:
                if 'processed' in c:
                    file = f'az://{a[0]}/{c}'
                    processed_list.append(pd.read_csv(file, storage_options=storage_options))
    df = pd.concat(processed_list)
    df.insert(0,'site_no',[i[:8] for i in df.sample_id])
    df.insert(0,'region',con)
    db.append(process_df(df, con))
db = pd.concat(db).replace('--', np.nan).dropna().reset_index(drop=True)

In [3]:
prefix_url = 'https://fluviusdata.blob.core.windows.net'
fout_scl = []
fout_rgb = []
for con in container_list:
    f = fs.walk(f'{con}-data/stations')
    processed_list = []
    for a in f:
        for b in a:
            for c in b:
                if '.tif' in c:
                    file = f'{prefix_url}/{a[0]}/{c}'
                    sample_id = file.split('/')[-1][:17]
                    if 'rgb' in file:
                        fout_rgb.append({'sample_id':sample_id,'rgb_href':file})
                    elif 'scl' in file:
                        fout_scl.append({'sample_id':sample_id,'scl_href':file})
                    #processed_list.append(pd.read_csv(file, storage_options=storage_options))

In [4]:
#link these files to the sample
df_rgb = pd.DataFrame(fout_rgb)
df_scl = pd.DataFrame(fout_scl)

In [5]:
db = db.merge(df_rgb.merge(df_scl))

In [37]:
#make the hrefs for image
from PIL import Image
import requests
from azure.storage.blob import BlobClient
from src.utils import url_to_img, local_to_blob

selections = ['rgb', 'scl']
out = []
container = 'app'
for selection in selections:
    out_png_href = []
    for i,r in db.iterrows():
        url = r[f'{selection}_href']
        #im = url_to_img(url)
        #tmpfile = '/tmp/img.png' 
        #im.save(tmpfile)
        blobname = f'img/{selection}/{os.path.basename(url).split(".")[0]}.png'
        #local_to_blob(container, tmpfile, blobname, storage_options)
        href = f'https://fluviusdata.blob.core.windows.net/{container}/{blobname}'
        out_png_href.append({'sample_id':r['sample_id'],\
                             f'{selection}_png_href':href})
    out.append(pd.DataFrame(out_png_href))
out = out[0].merge(out[1])

In [41]:
db = db.merge(out)

In [61]:
db = db.drop_duplicates()

In [62]:
db.to_json('az://modeling-data/fluvius_data.json',\
           storage_options=storage_options)
db.to_csv('az://modeling-data/fluvius_data.csv',\
          index=False,\
          storage_options=storage_options)
db.to_json('az://app/fluvius_data.json',\
          storage_options=storage_options)

In [47]:
pd.read_json('az://modeling-data/fluvius_data.json',\
           storage_options=storage_options)

,region,site_no,sample_id,julian,Date-Time,Q (m3/s),SSC (mg/L),Chip Cloud Pct,sentinel-2-l2a_R,sentinel-2-l2a_G,sentinel-2-l2a_B,rgb_href,scl_href,rgb_png_href,scl_png_href
0,ana,21750000,21750000_00000015,282,2016-10-08,27.151000,4.8,4.600694,49.656682,47.092166,39.410138,https://fluviusdata.blob.core.windows.net/ana-...,https://fluviusdata.blob.core.windows.net/ana-...,https://fluviusdata.blob.core.windows.net/app/...,https://fluviusdata.blob.core.windows.net/app/...
1,ana,22050001,22050001_00000030,269,2017-09-26,481.682000,1.5,2.332362,79.961310,91.555736,64.028950,https://fluviusdata.blob.core.windows.net/ana-...,https://fluviusdata.blob.core.windows.net/ana-...,https://fluviusdata.blob.core.windows.net/app/...,https://fluviusdata.blob.core.windows.net/app/...
2,ana,22050001,22050001_00000031,93,2019-04-03,851.221000,244.2,0.416493,60.305261,90.178330,62.198118,https://fluviusdata.blob.core.windows.net/ana-...,https://fluviusdata.blob.core.windows.net/ana-...,https://fluviusdata.blob.core.windows.net/app/...,https://fluviusdata.blob.core.windows.net/app/...
3,ana,22050001,22050001_00000032,211,2019-07-30,492.400000,1.5,0.166597,45.599346,58.495724,34.121479,https://fluviusdata.blob.core.windows.net/ana-...,https://fluviusdata.blob.core.windows.net/ana-...,https://fluviusdata.blob.core.windows.net/app/...,https://fluviusdata.blob.core.windows.net/app/...
4,ana,22700000,22700000_00000017,199,2018-07-18,133.829000,1.1,0.583090,16.975336,20.751121,15.405830,https://fluviusdata.blob.core.windows.net/ana-...,https://fluviusdata.blob.core.windows.net/ana-...,https://fluviusdata.blob.core.windows.net/app/...,https://fluviusdata.blob.core.windows.net/app/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3398,usgs,09326500,09326500_00008351,118,2019-04-28,4.615646,870.0,9.626039,35.092317,29.657993,18.603470,https://fluviusdata.blob.core.windows.net/usgs...,https://fluviusdata.blob.core.windows.net/usgs...,https://fluviusdata.blob.core.windows.net/app/...,https://fluviusdata.blob.core.windows.net/app/...
3399,usgs,09326500,09326500_00010016,188,2019-07-07,7.107529,120.0,99.238227,151.642857,145.535714,114.642857,https://fluviusdata.blob.core.windows.net/usgs...,https://fluviusdata.blob.core.windows.net/usgs...,https://fluviusdata.blob.core.windows.net/app/...,https://fluviusdata.blob.core.windows.net/app/...
3400,usgs,09326500,09326500_00010495,208,2019-07-27,2.491883,87.0,61.565097,78.930141,72.975667,59.923862,https://fluviusdata.blob.core.windows.net/usgs...,https://fluviusdata.blob.core.windows.net/usgs...,https://fluviusdata.blob.core.windows.net/app/...,https://fluviusdata.blob.core.windows.net/app/...
3401,usgs,09326500,09326500_00013015,168,2020-06-16,4.020992,63.0,14.819945,70.079439,59.373164,43.405874,https://fluviusdata.blob.core.windows.net/usgs...,https://fluviusdata.blob.core.windows.net/usgs...,https://fluviusdata.blob.core.windows.net/app/...,https://fluviusdata.blob.core.windows.net/app/...


In [71]:
scl_labels = [
    {'label':0,'class':'NO_DATA','color':'#000000'},
    {'label':1,'class':'SATURATED_OR_DEFECTIVE','color':'#ff0000'},
    {'label':2,'class':'DARK_AREA_PIXELS','color':'#404040'},
    {'label':3,'class':'CLOUD_SHADOWS','color':'#833c0c'},
    {'label':4,'class':'VEGETATION','color':'#00ff00'},
    {'label':5,'class':'NOT_VEGETATED','color':'#ffff00'},
    {'label':6,'class':'WATER','color':'#0000cc'},
    {'label':7,'class':'UNCLASSIFIED','color':'#757171'},
    {'label':8,'class':'CLOUD_MEDIUM_PROBABILITY','color':'#aeaaaa'},
    {'label':9,'class':'CLOUD_HIGH_PROBABILITY','color':'#d0cece'},
    {'label':10,'class':'THIN_CIRRUS','color':'#00ccff'},
    {'label':11,'class':'SNOW','color':'#ff66ff'}]
    

In [74]:
pd.DataFrame(scl_labels).to_json('az://app/scl_mapping.json',\
          storage_options=storage_options)

In [113]:
#concat all the metadata
container_list = ['ana', 'itv', 'usgsi', 'usgs']
#we are looking for all the individual processed csv
#adding a column for the data type
db = []
for con in container_list:
    f = pd.read_csv(f'az://{con}-data/{con}_station_metadata.csv',\
                   storage_options=storage_options)
    f.insert(0,'region',con)
    db.append(f)
df = pd.concat(db)
df['site_name'] = df['site_name'].fillna(df['station_name']).fillna(df['site_no'])
df = df.drop(columns=['station_name']).reset_index(drop=True)
#https://dados.gov.br/dataset?organization=agencia-nacional-de-aguas-ana

In [116]:
df.to_json('az://app/station_metadata.json',\
          storage_options=storage_options)